# Crop Labeled Screenshots
After running my 2560 × 1440 screenshots through the YOLO model, and cleaning up the tags, they need to be cropped in order to be used for training.

## Annotation-Aware Cropping
Crop around each bounding box with some padding to preserve context.

In [8]:
import os
import cv2

# Paths
image_folder = os.path.expanduser("~/Pictures/D4 capture 2024-11-06/scrubbing results (2024-11-15)")
label_folder = os.path.expanduser("~/Pictures/D4 capture 2024-11-06/scrubbing results (2024-11-15)")
output_folder = os.path.expanduser("~/Pictures/D4 capture 2024-11-06/scrubbing results (2024-11-15)/cropped")
os.makedirs(output_folder, exist_ok=True)

# Parameters
crop_size = 640
padding = 20  # Extra space around bounding boxes

# Process each image
for label_file in os.listdir(label_folder):
    if label_file.endswith('.txt'):
        base_name = os.path.splitext(label_file)[0]
        image_path = os.path.join(image_folder, f"{base_name}.png")
        label_path = os.path.join(label_folder, label_file)

        # Read image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading {image_path}. Skipping...")
            continue
        height, width, _ = image.shape

        # Read bounding boxes
        boxes = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                class_id, x_center, y_center, bbox_width, bbox_height = map(float, line.split())

                # Convert YOLO format to pixel coordinates
                x_center *= width
                y_center *= height
                bbox_width *= width
                bbox_height *= height

                x_min = int(x_center - bbox_width / 2)
                y_min = int(y_center - bbox_height / 2)
                x_max = int(x_center + bbox_width / 2)
                y_max = int(y_center + bbox_height / 2)

                boxes.append((class_id, x_min, y_min, x_max, y_max))

        # Create crops and labels
        crop_count = 0
        for class_id, x_min, y_min, x_max, y_max in boxes:
            box_width = x_max - x_min
            box_height = y_max - y_min

            # Scale down if the object is larger than the crop size
            scale_factor = max(box_width / crop_size, box_height / crop_size, 1)
            if scale_factor > 1:
                image = cv2.resize(image, (int(width / scale_factor), int(height / scale_factor)))
                height, width, _ = image.shape
                x_min = int(x_min / scale_factor)
                y_min = int(y_min / scale_factor)
                x_max = int(x_max / scale_factor)
                y_max = int(y_max / scale_factor)

            # Ensure the crop is 640x640
            crop_x_min = max(0, min(x_min - padding, width - crop_size))
            crop_y_min = max(0, min(y_min - padding, height - crop_size))
            crop_x_max = crop_x_min + crop_size
            crop_y_max = crop_y_min + crop_size

            # Extract crop
            crop = image[crop_y_min:crop_y_max, crop_x_min:crop_x_max]
            crop_filename = f"{base_name}_crop_{crop_count}.png"
            crop_path = os.path.join(output_folder, crop_filename)
            cv2.imwrite(crop_path, crop)

            # Adjust bounding boxes for the crop
            adjusted_boxes = []
            for cid, bx_min, by_min, bx_max, by_max in boxes:
                # Check if the box is within the crop
                if bx_max > crop_x_min and bx_min < crop_x_max and by_max > crop_y_min and by_min < crop_y_max:
                    # Adjust coordinates relative to the crop
                    adj_x_min = max(0, bx_min - crop_x_min)
                    adj_y_min = max(0, by_min - crop_y_min)
                    adj_x_max = min(crop_size, bx_max - crop_x_min)
                    adj_y_max = min(crop_size, by_max - crop_y_min)

                    # Convert adjusted coordinates to YOLO format
                    x_center = (adj_x_min + adj_x_max) / 2 / crop_size
                    y_center = (adj_y_min + adj_y_max) / 2 / crop_size
                    bbox_width = (adj_x_max - adj_x_min) / crop_size
                    bbox_height = (adj_y_max - adj_y_min) / crop_size
                    adjusted_boxes.append((cid, x_center, y_center, bbox_width, bbox_height))

            # Save adjusted labels to a new file
            label_filename = f"{base_name}_crop_{crop_count}.txt"
            label_path = os.path.join(output_folder, label_filename)
            with open(label_path, 'w') as lf:
                for cid, x_center, y_center, bbox_width, bbox_height in adjusted_boxes:
                    lf.write(f"{int(cid)} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

            crop_count += 1

print("Cropping and annotation adjustment completed!")

Cropping and annotation adjustment completed!
